In [1]:
import torch
from torchvision import datasets, transforms
from torchvision.transforms.functional import pad as PAD
import numbers
import numpy as np

import os

In [2]:
def get_padding(image):    
    w, h = image.size
    max_wh = np.max([w, h])
    h_padding = (max_wh - w) / 2
    v_padding = (max_wh - h) / 2
    l_pad = h_padding if h_padding % 1 == 0 else h_padding+0.5
    t_pad = v_padding if v_padding % 1 == 0 else v_padding+0.5
    r_pad = h_padding if h_padding % 1 == 0 else h_padding-0.5
    b_pad = v_padding if v_padding % 1 == 0 else v_padding-0.5
    padding = (int(l_pad), int(t_pad), int(r_pad), int(b_pad))
    return padding

class NewPad(object):
    def __init__(self, fill=0, padding_mode='constant'):
        assert isinstance(fill, (numbers.Number, str, tuple))
        assert padding_mode in ['constant', 'edge', 'reflect', 'symmetric']

        self.fill = fill
        self.padding_mode = padding_mode
        
    def __call__(self, img):
        """
        Args:
            img (PIL Image): Image to be padded.

        Returns:
            PIL Image: Padded image.
        """
        return PAD(img, get_padding(img), self.fill, self.padding_mode)
    
    def __repr__(self):
        return self.__class__.__name__ + '(padding={0}, fill={1}, padding_mode={2})'.\
            format(self.fill, self.padding_mode)

In [3]:
input_size=1024
data_transforms = {
    'train': transforms.Compose([
        NewPad(),
        transforms.Resize(input_size),
        transforms.ToTensor()
    ]),
}

In [4]:
data_dir = "/mnt/HDD3/Users/dasleo/data/MURA-v1.1/all/"
data_sets = {x:datasets.ImageFolder(os.path.join(data_dir,x),
                          data_transforms[x]) for x in ['train']}
dataloaders_dict = {x: torch.utils.data.DataLoader(data_sets[x],
                                           batch_size=20,
                                           shuffle=False,
                                           num_workers=2) for x in ['train']}

In [5]:
mean = 0.
std = 0.
total_steps = len(dataloaders_dict['train'].dataset)//20
for i, (images,classes) in enumerate(dataloaders_dict['train']):
    batch_samples = images.size(0) # batch size (the last batch can have smaller size!)
    images = images.view(batch_samples, images.size(1), -1)
    mean += images.mean(2).sum(0)
    std += images.std(2).sum(0)
    print(i/total_steps*100,end='\r')

mean /= len(dataloaders_dict['train'].dataset)
std /= len(dataloaders_dict['train'].dataset)

In [6]:
mean /= len(dataloaders_dict['train'].dataset)

In [7]:
mean

tensor([4.4784e-06, 4.4784e-06, 4.4784e-06])

In [8]:
std /= len(dataloaders_dict['train'].dataset)

In [9]:
std

tensor([4.1228e-06, 4.1228e-06, 4.1228e-06])

In [13]:
total_mean = mean*len(dataloaders_dict['train'].dataset)

In [14]:
total_mean

tensor([0.1523, 0.1523, 0.1523])

In [15]:
total_std = std * len(dataloaders_dict['train'].dataset)

In [16]:
total_std

tensor([0.1402, 0.1402, 0.1402])